In [36]:
import numpy as np
import pandas as pd
import soundfile as sf
import matplotlib.pyplot as plt
import librosa
import os
import tensorflow as tf
from keras import layers

In [38]:
data_train = pd.read_csv('dataset_train.csv')
data_test = pd.read_csv('dataset_test.csv')

data_test_folder = "./IRMAS-TestingData/" 

def getdParam():
    dList = []
    for filename in data_train['name']:
        path = os.path.join(data_test_folder, filename + ".wav")
        audio, sr = librosa.load(path)
        dList.append(librosa.get_duration(y=audio, sr=sr))
    return dList

dmin = 5.019773242630386 # La fonction prend 1min30 à s'executer, on a donc sauvegardé le résultat dans une variable
dmean = 16.886306332321844 
dmax = 20.0
len = 2874

def getSpectre():
    L = []

    for path in data_train['path']:
        audio, sr = librosa.load(path)
        spectre = librosa.feature.melspectrogram(y = audio, sr = sr, n_mels = 128, fmax = 8000)
        ptodb = librosa.power_to_db(spectre, ref=np.max)
        L.append(ptodb)
        
    list_spectres = np.array(L)
    np.save('list_spectres.npy', list_spectres)
    return list_spectres

#list_spectres = getSpectre() ##La fonction prend très longtemps à créer le fichier npy, on l'execute une fois et on enregistre le fichier
list_spectres = np.load('list_spectres.npy')
print(list_spectres.shape, data_train.shape)

x_train = list_spectres
colonnes_y = ["cel", "cla", "flu", "gac", "gel", "org", "pia", "sax", "tru", "vio", "voi", "drum"]
y_train = data_train[colonnes_y].values
print(x_train.shape, y_train.shape)

(6705, 128, 130) (6705, 14)
(6705, 128, 130) (6705, 12)


In [40]:
model = tf.keras.Sequential([
    layers.Input(shape=(128, 130)),  # Input layer with shape (128, 120)
    layers.Flatten(),                # Flatten the input into a 1D array
    layers.Dense(64, activation='relu'),   # Dense layer with 64 units and ReLU activation
    layers.Dense(12, activation='softmax')  # Output layer with 12 units and softmax activation
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, batch_size=32, epochs=5)

#loss, accuracy = model.evaluate(x_test, y_test)
#print("Test loss:", loss)
#print("Test accuracy:", accuracy)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 16640)             0         
                                                                 
 dense_4 (Dense)             (None, 64)                1065024   
                                                                 
 dense_5 (Dense)             (None, 12)                780       
                                                                 
Total params: 1,065,804
Trainable params: 1,065,804
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
210/210 [==============================] - 7s 27ms/step - loss: 32.2416 - accuracy: 0.1776
Epoch 2/5
210/210 [==============================] - 4s 19ms/step - loss: 3.3137 - accuracy: 0.2550
Epoch 3/5
210/210 [==============================] - 2s 9ms/step - loss: 3.2881 - accuracy: 0.2550
Epoch 4/